In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Lectura de data sets

In [ ]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Text preprocess

In [ ]:
!pip install symspellpy

In [ ]:
#Importamos librerias de pre procesamiento de texto
import nltk
import re
import string
from nltk.corpus import stopwords

from tqdm import tqdm
import spacy
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import json

In [ ]:
#utilizamos la libreria "re" para limpieza de texto
#uso funcion 
#re.sub(pattern, repl, string, count=0, flags=0)
#elimino corchetes, links, <>, puntuaciones, saltos de linea
#y remuevo las palabras que contienen numeros

#remuevo todas las puntuaciones salvo # y @
#no remuevo urls pq los pienso reemplazar mas adelante
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    punctuations_to_remove = string.punctuation.replace("#","")
    punctuations_to_remove = punctuations_to_remove.replace("@","")
    text = re.sub('[%s]' % re.escape(punctuations_to_remove), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

#Remuevo emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
def _lemmatize_text(sentence, nlp):
    sent = ""
    doc = nlp(sentence)
    for token in doc:
        if '@' in token.text:
            sent+=" @MENTION"
        elif '#' in token.text:
            sent+= " #HASHTAG"
        else:
            sent+=" "+token.lemma_
    return sent

In [ ]:
def simplify(sentence):
    sent = _replace_urls(sentence)
    sent = _simplify_punctuation(sent)
    sent = _normalize_whitespace(sent)
    return sent

def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "<URL>", text)
    return text

def _simplify_punctuation(text):
    """
    This function simplifies doubled or more complex punctuation. The exception is '...'.
    """
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1', corrected)
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

def _normalize_whitespace(text):
    """
    This function normalizes whitespaces, removing duplicates.
    """
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
def normalize_contractions(text, contractions):
    """
    This function normalizes english contractions.
    """
    new_token_list = []
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        first_upper = False
        if word[0].isupper():
            first_upper = True
        if word.lower() in contractions:
            replacement = contractions[word.lower()]
            if first_upper:
                replacement = replacement[0].upper()+replacement[1:]
            replacement_tokens = replacement.split()
            if len(replacement_tokens)>1:
                new_token_list.append(replacement_tokens[0])
                new_token_list.append(replacement_tokens[1])
            else:
                new_token_list.append(replacement_tokens[0])
        else:
            new_token_list.append(word)
    sentence = " ".join(new_token_list).strip(" ")
    return sentence

In [ ]:
def init_spellchecker():
    max_edit_distance_dictionary= 3
    prefix_length = 4
    spellchecker = SymSpell(max_edit_distance_dictionary, prefix_length)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spellchecker.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spellchecker.load_bigram_dictionary(dictionary_path, term_index=0, count_index=2)
    return spellchecker
    
def spell_correction(text, spellchecker):
    """
    This function does very simple spell correction normalization using pyspellchecker module. It works over a tokenized sentence and only the token representations are changed.
    """
    if len(text) < 1:
        return ""
    #Spell checker config
    max_edit_distance_lookup = 2
    suggestion_verbosity = Verbosity.TOP # TOP, CLOSEST, ALL
    #End of Spell checker config
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        if word is None:
            token_list[word_pos] = ""
            continue
        if not '\n' in word and word not in string.punctuation and not is_numeric(word) and not (word.lower() in spellchecker.words.keys()):
            suggestions = spellchecker.lookup(word.lower(), suggestion_verbosity, max_edit_distance_lookup)
            #Checks first uppercase to conserve the case.
            upperfirst = word[0].isupper()
            #Checks for correction suggestions.
            if len(suggestions) > 0:
                correction = suggestions[0].term
                replacement = correction
            #We call our _reduce_exaggerations function if no suggestion is found. Maybe there are repeated chars.
            else:
                replacement = _reduce_exaggerations(word)
            #Takes the case back to the word.
            if upperfirst:
                replacement = replacement[0].upper()+replacement[1:]
            word = replacement
            token_list[word_pos] = word
    return " ".join(token_list).strip()

def _reduce_exaggerations(text):
    """
    Auxiliary function to help with exxagerated words.
    Examples:
        woooooords -> words
        yaaaaaaaaaaaaaaay -> yay
    """
    correction = str(text)
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1', correction)

def is_numeric(text):
    for char in text:
        if not (char in "0123456789" or char in ",%.$"):
            return False
    return True


In [ ]:
def text_preprocessing(text, nlp, contractions, spellchecker):
    text = simplify(text)
    text = normalize_contractions(text, contractions)
    text = spell_correction(text, spellchecker)
    #la limpieza la hago al final dado 
    #que remueve todas las puntuaciones y muchas son importantes
    #para la normalizacion del texto como (')
    text = clean_text(text)
    text = _lemmatize_text(text, nlp).strip()
    return text

### Aplico funciones de preprocesamiento al data set

In [ ]:
#pre- procesamiento
df_train.head(10)

In [ ]:
nlp = spacy.load('en')
spellchecker = init_spellchecker()
contraction_list = json.loads(open('../input/contractions/english_contractions.json', 'r').read())    
#Train set
df_train['text'] = df_train['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))
#Test set
df_test['text'] = df_test['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))

In [ ]:
#post- procesamiento
df_train.head(30)

In [ ]:
df_test.head()

## GloVe Embeddings
Cargo el diccionario de embeddings con arrays de coeficientes que corresponde a cada palabra

In [ ]:
## Importo librerias 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Embedding
from keras.initializers import Constant

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D
from keras.optimizers import Adam

In [ ]:
embeddings_index = {}
f = open('/kaggle/input/glove6b/glove.6B.100d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vectors representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

### Tokenizacion con keras

In [ ]:
MAX_NUM_WORDS = 25000
MAX_SEQUENCE_LENGTH = 50

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(df_train['text'])
sequences = tokenizer.texts_to_sequences(df_train['text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen = MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)

In [ ]:
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1

#Inicializo la matriz con todos ceros
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word,i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Cargo la embedding_matrix en un Embedding layer. Seteo trainable=False 
para prevenir que los pesos sean actualizados durante el entrenamiento

In [ ]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [ ]:
model = Sequential()
#Al setear trainable false, en summary aparecen como not-trainable
model.add(embedding_layer)
model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(SpatialDropout1D(0.2))
model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

my_adam = Adam(learning_rate=5e-4)

model.summary()
model.compile(loss='binary_crossentropy', optimizer=my_adam, metrics=['accuracy'])

In [ ]:
#Fit train data
print(data.shape)
model.fit(data, df_train['target'], validation_split=0.2,batch_size=32, epochs = 4)

## Predict

In [ ]:
#Preparo el test set
test_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
test_tokenizer.fit_on_texts(df_test['text'])
test_sequences = test_tokenizer.texts_to_sequences(df_test['text'])
test_data = pad_sequences(test_sequences, maxlen = MAX_SEQUENCE_LENGTH)

In [ ]:
pred = model.predict(test_data)
pred = np.around(pred)
pred = pred.astype(int)

In [ ]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample_submission['target'] = pred
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head()

## Verifico submission.csv

In [ ]:
df_sub = pd.read_csv("./submission.csv")
df_test_copy = df_test.copy(deep=True)
df_test_copy.head()

In [ ]:
#Copio el target de la prediccion al data set de test
df_test_copy['target'] = df_sub['target']
df_test_copy.head(10)